In [ ]:
def isValid(s,a):


    return (s+a>=0 and s+a<l)
def tau(s,a):

    if(a==0):#up
        if(isValid(s[0],-1)):
            return (s[0]-1,s[1])
    elif(a==1):#down
        if(isValid(s[0],+1)):
            return (s[0]+1,s[1])
    elif(a==2):#left
        if(isValid(s[1],-1)):
            return (s[0],s[1]-1)    
    elif(a==3):#right
        if(isValid(s[1],1)):
            return (s[0],s[1]+1)    
    return(s)